#IMPORT DEPENDENCIES

---



In [ ]:
# Import keras packages
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
# Import keras packages
import keras
from tensorflow.keras import applications 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense , Conv2D , MaxPooling2D , Flatten , GlobalAveragePooling2D , Dropout
from keras.models import Sequential
import tensorflow as tf 
from tensorflow import keras  #to load model after save
from keras.models import load_model
# pour calculer la matrice de confusion
from sklearn.metrics import confusion_matrix

# pour réduisez le taux d'apprentissage lorsqu'une métrique a cessé de s'améliorer.
from keras.callbacks import ReduceLROnPlateau

# pour pouvoir importer utiliser le modèle pré-entraîner VGG19
from tensorflow.keras.applications import VGG19

# pour convertit le vecteur (y) de classe (entiers) en matrice de classe binaire.
from tensorflow.keras.utils import to_categorical

# pour générer des images altere
from keras.preprocessing.image import ImageDataGenerator

# pour initialise le methode de le desente gradien
from tensorflow.keras.optimizers import SGD, Adam

from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense, Dropout

Connexion to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


check the distribution of the images in the database

In [ ]:
def walk_through_dir(directory_name):
  for dirpaths,dirnames,filenames in os.walk(directory_name):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpaths}'")

In [ ]:
input_data_dir= '/content/drive/MyDrive/drive-download-20220816T231016Z-001' #Dataset URL

Verifiy classes number

In [ ]:
walk_through_dir(input_data_dir)

There are 4 directories and 0 images in '/content/drive/MyDrive/drive-download-20220816T231016Z-001'
There are 0 directories and 939 images in '/content/drive/MyDrive/drive-download-20220816T231016Z-001/Wheat Loose Smut'
There are 0 directories and 1030 images in '/content/drive/MyDrive/drive-download-20220816T231016Z-001/Healthy Wheat'
There are 0 directories and 696 images in '/content/drive/MyDrive/drive-download-20220816T231016Z-001/Crown and Root Rot'
There are 0 directories and 849 images in '/content/drive/MyDrive/drive-download-20220816T231016Z-001/Leaf Rust'


In [ ]:
import glob as gb
import glob
import numpy as np
import pandas as pd


In [ ]:
def get_dataCategories(input_data_dir):
    categories = []
    for folder_name in os.listdir(input_data_dir):
        if os.path.isdir(os.path.join(input_data_dir, folder_name)):
            nbr_files = len(
                glob.glob(os.path.join(input_data_dir, folder_name) + "/**")
            )
            categories.append(np.array([folder_name, nbr_files]))

    categories.sort(key=lambda a: a[0])
    cat = np.array(categories)

    return list(cat[:, 0]), list(cat[:, 1])




categories, nbr_files = get_dataCategories(input_data_dir)

# Create DataFrame
df = pd.DataFrame({"categorie": categories, "numbre of files": nbr_files})
print("number of categories: ", len(categories))
df




*  Read the images from the dataset 
*  Resize them using the dimensions img_wid, img_high
*  Create the set of features **X** and the labels **y**



In [ ]:
import cv2
def create_dataset(input_data_dir, categories, img_wid, img_high):
    X, y = [], []
    for category in categories:
        path = os.path.join(input_data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                ima_resize_rgb = cv2.resize(img_array, (img_wid, img_high))

                X.append(ima_resize_rgb)
                y.append(class_num)

            except Exception as e:
                pass

    y = np.array(y)
    X = np.array(X).reshape(y.shape[0], img_wid, img_wid, 3)
    return X, y

img_wid, img_high = 300, 300
X, y = create_dataset(input_data_dir, categories, img_wid, img_high)

print(f"X: {X.shape}")
print(f"y: {y.shape}")


X: (144, 300, 300, 3)
y: (144,)


Convert **y** to scaler format

In [ ]:
Y = np.reshape(y, (len(y), 1))

Split dataset to train and test set


In [ ]:
#il faut l'optimiser 
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, train_size=0.8
)
print(f"X_train: {X_train.shape}")
print(f"t_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

In [ ]:
# define training and test sets
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
x_test = X_test

# Dimension of the dataset
print(f"x_train:{x_train.shape},  y_train:{y_train.shape}")
print(f"x_train:{x_val.shape},  y_train:{y_val.shape}")
print(f"x_train:{x_test.shape},  y_train:{y_test.shape}")

In [ ]:
from keras.utils.np_utils import to_categorical

# One Hot Encoding
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

# Verifying the dimension after one hot encoding
print(f"x_train:{x_train.shape},  y_train:{y_train.shape}")
print(f"x_val:{x_val.shape},  y_val:{y_val.shape}")
print(f"x_test:{x_test.shape},  y_test:{y_test.shape}")

In [ ]:
# Image Data Augmentation
train_generator = ImageDataGenerator(
  rotation_range=30,
  zoom_range=0.15,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.15,
  horizontal_flip=True,
  fill_mode="nearest")

val_generator = ImageDataGenerator(
    rotation_range=30, horizontal_flip=True, zoom_range=0.15
)

test_generator = ImageDataGenerator(
    rotation_range=30, horizontal_flip=True, zoom_range=0.15
)

# Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

#TRAIN

In [ ]:
number_of_classes =  4
batch_size=32

In [ ]:
import os

import numpy as np
#import tensorflow as tf
import matplotlib.pyplot as plt
from glob import glob

from tqdm import tqdm_notebook
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import load_model, Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

%matplotlib inline
from keras.layers import AveragePooling2D 


In [ ]:
headmodel = VGG19(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(300, 300, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size=(5, 5))(model)
model = Flatten(name="flatten")(model)
model = Dense(512, activation="relu")(model)
model = Dropout(0.2)(model)
model = Dense(3, activation="softmax")(model)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
moodel = Model(inputs=headmodel.input, outputs=model)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in headmodel.layers:
    layer.trainable = False

In [ ]:
moodel.summary()

In [ ]:
opt = Adam(lr=1e-4)
moodel.compile(loss="categorical_crossentropy", optimizer=opt,
               metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# initialized with actual "learned" values versus pure random
H = moodel.fit(
    train_generator.flow(x_train, y_train, batch_size=16),
    steps_per_epoch=len(x_train) // 16,
    validation_data=val_generator.flow(x_val,y_val),
    validation_steps=len(x_val) // 16,
    epochs=20)

Plot accuracy & loss

In [ ]:
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('model 4 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('model 4 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

Evaluate the model

In [ ]:
loss, acc = moodel.evaluate(x_test, y_test)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

1/1 [==============================] - 5s 5s/step - loss: 0.8481 - accuracy: 0.7931
Untrained model, accuracy: 79.31%


In [ ]:
moodel.save('moodelvgg81.h5')

#Load saved model

Load model

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/moodelvgg81.h5')


Matrice De Confusion

In [ ]:
"""
    Les fonctions utilise affiche la matrice de confusion
"""

# ----------------------------------------------------------------------
def cm_plt(ax, cm, classes, cmap, title, normalize):
    """
       affiche la matrice de confusion appartire de l'ax entre

    Args:
        ax (plt): ax qui sert a affiche la matrice
        cm (numpy): matrice de confusion
        classes (list): liste des classes
        cmap (plt): couluer de la matrice
        title (str): titre de la matrice
        normalize (booleen): vrai affichre cm normalisee

    Returns:
        plt: l'ax qui sera affiche
    """
    im = ax.imshow(cm, interpolation="nearest", cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=classes,
        yticklabels=classes,
        title=title,
        ylabel="True label",
        xlabel="Predicted label",
    )

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,
                i,
                format(cm[i, j], fmt),
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
            )

    return ax


# ----------------------------------------------------------------------
# Defining function for confusion matrix plot
def plt_confusion_mat(cm, classes, fig_size, cmap=plt.cm.Blues):
    """
        afficher la cm normalisee et non normalisee

    Args:
        cm (numpy): matrice de confusion
        classes (list): liste des classes
        fig_size (_type_): _description_
        cmap (plt, optional): couluer de la matrice. Defaults to plt.cm.Blues.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=fig_size)
    ax1 = cm_plt(
        ax1,
        cm,
        classes,
        cmap,
        title="Confusion matrix, without normalization",
        normalize=False,
    )

    cmn = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    ax2 = cm_plt(
        ax2,
        cmn,
        classes,
        cmap,
        title="Normalized confusion matrix",
        normalize=True,
    )


In [ ]:
# Making prediction
y_pred = np.argmax(moodel.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)

Plot Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

Plot Confusion matrix

In [ ]:
# get confusion matrix
confuision_mat = confusion_matrix(y_true, y_pred)
# plot confusion_mat
plt_confusion_mat(confuision_mat, classes=categories, fig_size=(20, 7))
